## Import

In [20]:
from tqdm import tqdm

import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import xlwings as xw
import investpy
import datetime
import os, re

## Load data

In [2]:
data_path = os.getcwd() + "\\datasets\\"

In [3]:
def read_xlsx(name):
    instance = xw.App(visible = False)
    xlsx_data = xw.Book(name).sheets[0]
    df = xlsx_data.range('A1').options(pd.DataFrame, index = False, expand = 'table').value
    instance.quit()
    instance.kill()
    return df

def read_data(name):
    try :
        output = pd.read_excel(name)
    except ValueError:    
        output = read_xlsx(name)
    return output

In [4]:
def get_stock_list():
    if os.path.isfile(data_path + "df_krx.xlsx") & os.path.isfile(data_path + "df_america.xlsx"):
        df_krx = read_data(data_path + "df_krx.xlsx")
        df_america = read_data(data_path + "df_america.xlsx")
        
    else:
        df_krx = fdr.StockListing('KRX')
        df_sp500 = fdr.StockListing('S&P500')
        df_nas = fdr.StockListing('NASDAQ')
        df_nyse = fdr.StockListing('NYSE')
        df_amex = fdr.StockListing('AMEX')

        df_america = pd.concat([df_nas, df_nyse, df_amex]).drop_duplicates()
        append_code = [x for x in df_sp500.Symbol if x not in list(df_america.Symbol)]
        append_df = df_sp500.loc[df_sp500.Symbol.isin(append_code)]
        del append_df['Sector']
        df_america = pd.concat([df_america, append_df])
        df_america.index = [x for x in range(df_america.shape[0])]
        
        df_krx.to_excel(data_path + "df_krx.xlsx", index = False)
        df_america.to_excel(data_path + "df_america.xlsx", index = False)
    
    return df_krx, df_america

In [59]:
# data 불러오기
if os.path.isfile(data_path + "df_infos.xlsx"):
    df_infos = read_data(data_path + "df_infos.xlsx")
    df_krx, df_america = get_stock_list()
    df_america = df_america.loc[:, ['Symbol', 'Name']]
    df_krx = df_krx.loc[:, ['Symbol', 'Name']]
else:
    df_krx, df_america = get_stock_list()
    df_america = df_america.loc[:, ['Symbol', 'Name']]
    df_krx = df_krx.loc[:, ['Symbol', 'Name']]
    df_infos = pd.concat([df_krx, df_america])
    df_infos = df_infos.dropna()
    df_infos.index = [x for x in range(df_infos.shape[0])]
    
    symbol_list = []

    for symbol, name in zip(df_infos.Symbol, df_infos.Name):
        if " PR " in symbol:
            symbol_list.append(re.sub(" PR ","/",symbol))
        elif (".U" in symbol) and ("Units" in name):
            symbol_list.append(re.sub(".U","/UN",symbol))
        elif "\." in symbol:
            symbol_list.append(re.sub("\.", "/", symbol))
        elif " RT" in symbol:
            symbol_list.append(symbol.split(" ")[0])
        elif symbol == 'BRKB':
            symbol_list.append("BRK/B")
        elif symbol == 'BFB':
            symbol_list.append("BF/B")
        else:
            symbol_list.append(symbol)

    df_infos['WINK_symbols'] = symbol_list
    df_infos.to_excel(data_path + "df_infos.xlsx", index = False)

* 환율 갱신

In [12]:
# data 불러오기
if os.path.isfile(data_path + "currency.xlsx"):
    currency = read_data(data_path + "currency.xlsx")

start_date = np.max(currency.Date)
try :
    append_curr = fdr.DataReader("USD/KRW", start = start_date)
    append_curr = append_curr.reset_index()

    currency = currency.loc[currency.Date < start_date, :]
    currency = pd.concat([currency, append_curr])
    currency.index = [x for x in range(currency.shape[0])]
    currency.to_excel(data_path + "currency.xlsx", index = False)
    
except:
    print("추가행 없음")
    pass

추가행 없음


* datasets에 데이터 있는지 확인

* 있으면 append

* 없으면 전체 수집

* 수집할 때 원화는 그대로 수집
* 외화는 환율 곱해서 원화로 변경

In [16]:
if os.path.isfile(data_path + "stock_df.xlsx"):
    stock_df = read_data(data_path + "stock_df.xlsx")
    start_date = np.max(stock_df.Date)
    
    # 추가될 데이터
    origin_df = stock_df.loc[stock_df.Date < start_date, :]
    origin_df.tail()
    
else:
    print("기존 데이터 없음")
    start_date = None

기존 데이터 없음


In [17]:
# symbols
symbol_list = list(df_infos.Symbol.unique())

In [18]:
def change_name(symbol, df, opt = True):
    if opt:
        output = df.loc[df.Symbol == symbol, 'Name'].values[0]
    else:
        output = df.loc[df.Symbol == symbol, 'WINK_symbols'].values[0]
    return output

def make_df(name, df):
    temp_df = pd.DataFrame(df.Close)
    temp_df.columns = ['{}'.format(name)]
    return temp_df.reset_index()

* 국내는 이름
* 해외는 심볼

In [73]:
start_idx = 0
save_cnt = 0
check_idx = 0
option = True

In [74]:
if not os.path.isdir(data_path + "parts"):
    os.mkdir(data_path + "parts")
part_path = data_path + "parts\\"

In [81]:
try:
    for idx, symbol in enumerate(tqdm(symbol_list[start_idx:])):
        try:
            df = fdr.DataReader(symbol, start = start_date)

        except ValueError:
            start_idx += 1
            continue

        if df.shape[0] == 0:
            start_idx += 1
            continue

        if start_idx > df_krx.shape[0]:
            option = False    

        name = change_name(symbol, df_infos, option)

        if check_idx == 0 :
            stock_df = make_df(name, df)
        else:
            temp_df = make_df(name, df)
            stock_df = pd.merge(stock_df, temp_df, on ='Date', how = 'outer')

        check_idx += 1
        start_idx += 1

        if (check_idx % 100 == 0) and (check_idx > 10):
            stock_df.to_excel(part_path + "stock_df_{}.xlsx".format(save_cnt), index = False)
            del stock_df

            check_idx = 0
            save_cnt += 1
            time.sleep(10)

    stock_df.to_excel(part_path + "stock_df_{}.xlsx".format(save_cnt), index = False)

except:
    for idx, symbol in enumerate(tqdm(symbol_list[start_idx:])):
        try:
            df = fdr.DataReader(symbol, start = start_date)

        except ValueError:
            start_idx += 1
            continue

        if df.shape[0] == 0:
            start_idx += 1
            continue

        if start_idx > df_krx.shape[0]:
            option = False    

        name = change_name(symbol, df_infos, option)

        if check_idx == 0 :
            stock_df = make_df(name, df)
        else:
            temp_df = make_df(name, df)
            stock_df = pd.merge(stock_df, temp_df, on ='Date', how = 'outer')

        check_idx += 1
        start_idx += 1

        if (check_idx % 100 == 0) and (check_idx > 10):
            stock_df.to_excel(part_path + "stock_df_{}.xlsx".format(save_cnt), index = False)
            del stock_df

            check_idx = 0
            save_cnt += 1
            time.sleep(10)

    stock_df.to_excel(part_path + "stock_df_{}.xlsx".format(save_cnt), index = False)

 13%|██████████                                                                    | 1237/9542 [05:39<23:51,  5.80it/s]

HTTP 429 error. Retry after 12361 seconds (3:26:01)
HTTP 429 error. Retry after 12361 seconds (3:26:01)


 13%|██████████▏                                                                   | 1239/9542 [05:40<29:42,  4.66it/s]

HTTP 429 error. Retry after 12361 seconds (3:26:01)


 13%|██████████▏                                                                   | 1240/9542 [05:40<32:35,  4.25it/s]

...

HTTP 429 error. Retry after 12238 seconds (3:23:58)


  0%|▎                                                                               | 34/7994 [00:10<41:51,  3.17it/s]

HTTP 429 error. Retry after 12238 seconds (3:23:58)


  0%|▎                                                                               | 34/7994 [00:10<42:43,  3.10it/s]


KeyboardInterrupt: 

In [ ]:
stock_df.head()

* 예외 케이스

In [86]:
'''
except_symbol = []
except_name = []

for idx, symbol in enumerate(symbol_list[2091]):
    try:
        df = fdr.DataReader(symbol)
        
    except ValueError:
        name = change_name(symbol, df_krx)
        except_symbol.append(symbol)
        except_name.append(name)
        
    if df.shape[0] == 0:
        except_symbol.append(symbol)
        except_name.append(name)

np.save("except_symbol.npy", except_symbol)
np.save("except_name.npy", except_name)
'''